In [7]:
import numpy as np
import pandas as pd
import pandas_ta as ta

from plotly import graph_objects as go
from tqdm import tqdm

import logging

log = logging.Logger(name="logger", level=logging.INFO)

In [8]:
xau = pd.read_csv("data/xauusd_1h.csv", index_col="datetime", parse_dates=['datetime'])
xau.head()

,open,high,low,close,volume
datetime,,,,,
2010-01-03 22:00:00,1097.880005,1097.880005,1097.449951,1097.449951,2
2010-01-03 23:00:00,1097.680054,1100.599976,1095.979980,1095.979980,919
2010-01-04 00:00:00,1096.010010,1096.869995,1093.449951,1094.839966,1596
2010-01-04 01:00:00,1094.869995,1095.959961,1094.239990,1095.699951,869
2010-01-04 02:00:00,1095.670044,1099.150024,1095.630005,1098.329956,1054


In [9]:
xau.index[0]

Timestamp('2010-01-03 22:00:00')

In [10]:
def pivot_high(highs:pd.Series, window:int=14):
    len = highs.size
    ph = np.zeros(shape=(len, ), dtype=float)
    
    for i in range(window, len-window):
        if highs.iloc[i] == highs.iloc[i-window:i+window].max():
            ph[i] = highs.iloc[i]
    return ph

def pivot_low(lows:pd.Series, window:int=14):
    len = lows.size
    pl = np.zeros(shape=(len, ), dtype=float)
    
    for i in range(window, len-window):
        if lows.iloc[i] == lows.iloc[i-window:i+window].min():
            pl[i] = lows.iloc[i]
    return pl

In [11]:
def pivots(highs:pd.Series, lows:pd.Series, window:int) -> pd.Series:
    len = highs.size
    if lows.size != len:
        raise BufferError("`highs` and `lows` must have the same size!")
    pivots = pd.Series(data=0, index=highs.index)
    last_pivot = -1
    last_pivot_index = 0
    for i in tqdm(range(window, len-window)):
        if highs.iloc[i] == highs.iloc[i-window:i+window].max():
            if last_pivot == 1 and highs.iloc[last_pivot_index] < highs.iloc[i]:     # Previous pivot is a lower pivot high?!
                pivots.iloc[last_pivot_index] = 0       # Invalidate the previous pivot high
                last_pivot_index = i                    # Set new pivot high
                pivots.iloc[last_pivot_index] = last_pivot
            elif last_pivot == -1:  # Previous pivot is a pivot low?!
                last_pivot_index = i            # Set pivot high
                last_pivot = 1
                pivots.iloc[last_pivot_index] = last_pivot
        elif lows.iloc[i] == lows.iloc[i-window:i+window].min():
            if last_pivot == -1 and lows.iloc[last_pivot_index] > lows.iloc[i]:    # Previous pivot is a higher pivot low?!
                pivots.iloc[last_pivot_index] = 0       # Invalidate the previous pivot low
                last_pivot_index = i                    # Set new pivot low
                pivots.iloc[last_pivot_index] = last_pivot
            elif last_pivot == 1:       # Previous pivot is a pivot high
                last_pivot_index = i            # Set pivot low
                last_pivot = -1
                pivots.iloc[last_pivot_index] = last_pivot
    return pivots

In [12]:
xau['pivots'] = pivots(xau['high'], xau['low'], window=3)

100%|██████████| 89874/89874 [00:06<00:00, 13319.99it/s]


In [13]:
xau[xau['pivots'] == -1]['pivots'].size

7588

In [14]:
def plot_data(df):
    fig = go.Figure(
        data=[go.Candlestick(x=df.index,
                            close=df['close'],
                            open=df['open'],
                            low=df['low'],
                            high=df['high'])]
    )

    fig.update_layout(title="XAUUSD - H1",
                    xaxis_title="Date Time",
                    yaxis_title="Price",
                    xaxis=dict(type="category"))

    fig.show()

In [15]:
xau[xau['pivots'] == 1].index



DatetimeIndex(['2010-01-04 14:00:00', '2010-01-05 12:00:00',
               '2010-01-06 04:00:00', '2010-01-06 09:00:00',
               '2010-01-06 19:00:00', '2010-01-07 06:00:00',
               '2010-01-07 14:00:00', '2010-01-08 07:00:00',
               '2010-01-11 00:00:00', '2010-01-11 13:00:00',
               ...
               '2024-10-09 11:00:00', '2024-10-10 03:00:00',
               '2024-10-10 14:00:00', '2024-10-11 06:00:00',
               '2024-10-11 16:00:00', '2024-10-14 06:00:00',
               '2024-10-14 20:00:00', '2024-10-15 16:00:00',
               '2024-10-16 13:00:00', '2024-10-17 02:00:00'],
              dtype='datetime64[ns]', name='datetime', length=7589, freq=None)

In [16]:
# Store pivot values
xau['pivot_values'] = [xau['high'].iloc[i] if xau['pivots'].iloc[i] else xau['low'].iloc[i] if xau['pivots'].iloc == -1 else 0 for i in range(xau.shape[0])]
# xau['pivot_values'].loc[xau['pivots'] == 1] = xau['high'
# xau['pivot_values'] = xau['low'].loc[xau['pivots'] == -1]

In [17]:
xau['pivot_values'].loc[xau['pivot_values'] != 0]

datetime
2010-01-04 14:00:00    1123.920044
2010-01-04 18:00:00    1118.800049
2010-01-05 12:00:00    1128.859985
2010-01-05 18:00:00    1121.550049
2010-01-06 04:00:00    1124.875000
                          ...     
2024-10-15 16:00:00    2668.965088
2024-10-16 00:00:00    2660.610107
2024-10-16 13:00:00    2685.465088
2024-10-16 16:00:00    2674.260010
2024-10-17 02:00:00    2684.939941
Name: pivot_values, Length: 15177, dtype: float64

In [18]:
def head_and_shoulders(df:pd.DataFrame, threshold:int, log:logging.Logger) -> pd.Series:
    # if ['pivots', 'open', 'close'] not in df.columns.to_list():
        # raise ValueError("The columns `['pivots', 'open', 'close']` must exist...")

    buy_sell = pd.Series(data=0, index=df.index)
    len = df.shape[0]
    print(len, end="\n\n\n")
    pip_value = 0.1
    
    pivots_iloc = np.where(df['pivots'] != 0)[0]
    pivots_index = df.iloc[pivots_iloc].index
    
    for i in tqdm(range(len)):
        prev_5_pivots_index = pivots_index[pivots_index < df.index[i]]
        # print(prev_5_pivots_index)
        if prev_5_pivots_index.size < 5:
            continue
        else:
            prev_5_pivots_index = prev_5_pivots_index[-5:]
        prev_5_pivots = df['pivots'].loc[prev_5_pivots_index]
        prev_5_pivot_values = df['pivot_values'].loc[prev_5_pivots_index]

        if prev_5_pivots.loc[prev_5_pivots_index[0]] == 1:
            log.info("Start pivot high on left shoulder!")
            # if np.abs(prev_5_pivot_values.iloc[0] - prev_5_pivot_values.iloc[4]) < threshold * pip_value:
            shoulders = prev_5_pivot_values.iloc[0]
            if np.abs(prev_5_pivot_values.iloc[1] - prev_5_pivot_values.iloc[3]) < threshold * pip_value:
                neck_line = prev_5_pivot_values.iloc[1]
                if prev_5_pivot_values.iloc[2] > prev_5_pivot_values.iloc[0]:
                    if df['close'].iloc[i-1] < neck_line and df['open'].iloc[i-1] > neck_line:
                        buy_sell.iloc[i] = -1   # Sell
            
    return buy_sell

In [19]:
log.setLevel(logging.DEBUG)
buy_sell = head_and_shoulders(xau, 5, log=log)

89880




100%|██████████| 89880/89880 [00:37<00:00, 2419.55it/s]


In [55]:
xau['buy_sell'] = buy_sell

buy_sell.loc[buy_sell != 0]

datetime
2010-03-01 18:00:00   -1
2010-03-01 21:00:00   -1
2010-03-01 23:00:00   -1
2010-03-08 01:00:00   -1
2010-03-19 14:00:00   -1
                      ..
2023-09-20 01:00:00   -1
2024-01-24 15:00:00   -1
2024-02-28 07:00:00   -1
2024-06-21 13:00:00   -1
2024-09-06 14:00:00   -1
Length: 114, dtype: int64

In [56]:
is_sell = np.where(buy_sell == -1)[0]

i = 4
plot_data(xau.iloc[is_sell[i]-50:is_sell[i]+50])

In [59]:
xau_cap = xau.rename(columns={'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close':'Close',
            'volume': 'Volume'})

xau_cap.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'pivots', 'pivot_values',
       'buy_sell'],
      dtype='object')

In [102]:
from backtesting.backtesting import Strategy

def SIGNAL():
    return xau.buy_sell


class HnSStrategy(Strategy):       
    sl_points = 100
    point_size = .01
    rrr = 2.0
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        if self.signal == -1:
            sl = self.data.Close[-1] + self.sl_points * self.point_size
            print(self.data.Close[-1])
            tp = self.data.Close[-1] - self.rrr * self.sl_points * self.point_size
            self.sell(size=0.001, sl=sl, tp=tp)
    

In [103]:
from backtesting.backtesting import Backtest

# hns = HnSStrategy(xau_cap, sl_points=50, rrr=2.0)

params = {'sl_points': 50,
          'rrr': 2.0}

bt = Backtest(xau_cap, HnSStrategy, cash=200_000, commission=.02)

bt.run()

bt.plot()

1119.2550048828125


ValueError: Short orders require: TP (1117.2550048828125) < LIMIT (1096.8699047851562) < SL (1120.2550048828125)